# Results and future work

The APIs in total got about 500 jobs and the web scraping with selenium yeilded about 1300, so that isn't the most amount of dta but its enough to get some general mertrics and insights, plus the amount of work to unify this will be pretty substancial as well.

Getting the job descriptions more generalized to get a better feel for the data.


In [11]:
import pandas as pd

def group_job_title(title):
    title = str(title).lower()
    
    if 'hadoop' in title or 'data' in title or 'database' in title or 'spark' in title:
        return 'data engineer'
    elif 'azure' in title or 'aws' in title or 'cloud' in title:
        return 'cloud enginner'
    if 'full' in title or 'stack' in title or 'frontend' in title or 'backend' in title or 'front' in title or 'end' in title or 'react' in title or 'angular' in title or '.net' in title or 'node' in title or 'nodejs' in title:
        return 'web developer'
    elif 'machine learning' in title or 'ai' in title or 'ml' in title or 'quantitative' in title:
        return 'data science'
    elif 'network' in title or 'security' in title or 'cyber' in title or 'Cyber' in title or 'Cybersecurity' in title:
        return 'network engineer'
    elif 'andriod' in title or 'ios' in title:
        return 'mobile developer'
    elif 'nan' in title or 'null' in title:
        return 'no title given'
    elif 'c++' in title or 'C++' in title or 'java' in title or 'Java' or 'golang' in title or 'python' in title:
        return 'software engineer'
    else:
        return 'software egnineer'

# Read the Excel files
df1 = pd.read_excel('combined_indeed_jobs.xlsx')
df2 = pd.read_excel('indeed_jobs_250.xlsx',sheet_name='cleaned_data')
df3 = pd.read_excel('job_data_updated.xlsx')

# Apply the job category mapping to each DataFrame
df1['Category'] = df1['job_title_lower'].apply(group_job_title)
df2['Category'] = df2['job_title_lower'].apply(group_job_title)
df3['Category'] = df3['title_lower'].apply(group_job_title)

# # Print the updated DataFrames
# print(df1.head())
# print(df2.head())
# print(df3.head())

# Save the DataFrames with the new column back to Excel
df1.to_excel('combined_indeed_jobs_updated.xlsx', index=False)
df2.to_excel('indeed_jobs_250_updated.xlsx', index=False)
df3.to_excel('job_data_updated_new.xlsx', index=False)


Second thing I wanted to find was averaging the salary range out.  Since some data is missing for salaries, if I generalize the categories, I can take the average of those same categories for the position.  This might later turn into getting the numbers based on geo-location, but one step at a time.

In [27]:
import pandas as pd
import numpy as np
import re

# Define the salary extraction function
def extract_average_salary(salary):
    if pd.isna(salary):
        return None
    else:
        cleaned_salary = re.sub(r'[^\d.,-]', '', salary)
        numbers = re.findall(r'\d+(?:,\d+)?(?:\.\d+)?', cleaned_salary)
        
        if numbers:
            if len(numbers) == 2:
                # Extract the two numbers, remove commas, and calculate the average
                num1 = numbers[0].replace(',', '')
                num2 = numbers[1].replace(',', '')
                average = (float(num1) + float(num2)) / 2
                return average
            elif len(numbers) == 1:
                # Return the single given number
                return float(numbers[0].replace(',', ''))
        else:
            return None

# Read the Excel file
df1 = pd.read_excel('job_data_updated_new.xlsx')

# Apply the salary extraction function to the 'salary' column
df1['average_salary'] = df1['salary'].apply(extract_average_salary)

# Print the 'average_salary' column
df1.to_excel('job_data_updated_new.xlsx', index=False)


In [43]:
import pandas as pd
import locale

# Set the locale for formatting
locale.setlocale(locale.LC_ALL, '')

# Read the Excel file
df = pd.read_excel('job_data_updated_new.xlsx')

# Apply the formatting to the 'average_salary' column
df['average_salary_fixed'] = df['average_salary'].apply(lambda x: None if pd.isna(x) else locale.format_string("%.2f", float(x) * 1000, grouping=True))

# Print the updated DataFrame
print(df['average_salary_fixed'])

# Save the DataFrame with the new column back to Excel
df.to_excel('job_data_updated_new.xlsx', index=False)


0       102,950.00
1       102,050.00
2        68,900.00
3             None
4             None
           ...    
1558          None
1559          None
1560    122,500.00
1561    127,000.00
1562    133,500.00
Name: average_salary_fixed, Length: 1563, dtype: object


## Working with Power BI

Power BI offers great functionality like power query and transforming the data to get this into one big dataset without compromising the data, and keeping things easy

I was also trying to work to getting the location data standardized becauase aggregating would be difficult if there wasn't any way to have things flushed out

In [54]:
import pandas as pd

# Read the Excel file
data = pd.read_excel('combined_all_jobs.xlsx')
state_ref = pd.read_excel('state_ref_table.xlsx')

# Function to find the state from location_clean column
def find_state(location):
    for index, row in state_ref.iterrows():
        if row['State Code'] in location or row['State'] in location:
            return row['State']
    return None

# Apply the function to create the 'State' column
data['State'] = data['location_clean'].apply(find_state)

# Save the result with the added 'State' column
data.to_excel('combined_all_jobs.xlsx', index=False)
